In [46]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)


/home/utilisateur/projet/Brief_Prêt/Model_pret/Model/../Module/loader_csv.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_partie4 = pd.read_csv(partie4)


In [47]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[2806494000, 2032285009, 2423745002, 753706300..."
1,Name,object,14,0.00,779583,"[SMALL WONDERS CHILD DEVELOMENT, MOTT'S MINIAT..."
2,City,object,30,0.00,32581,"[BURBANK, CEDAR RAPIDS, CENTRAL POINT, SAN FRA..."
3,State,object,14,0.00,51,"[MD, MT, TX, NV, MO]"
4,Zip,int64,0,0.00,33611,"[43207, 48917, 95437, 55976, 23663]"
5,Bank,object,1559,0.17,5802,"[WELLS FARGO BANK NATL ASSOC, BANK OF AMERICA ..."
6,BankState,object,1566,0.17,56,"[AZ, CA, TX, OH, RI]"
7,NAICS,int64,0,0.00,1312,"[621210, 453998, 423910, 445299, 424940]"
8,ApprovalDate,object,0,0.00,9859,"[27-Mar-01, 5-Apr-90, 27-May-92, 22-Sep-92, 28..."
9,ApprovalFY,object,0,0.00,71,"[2001, 2003, 1996, 2011, 2009]"


In [48]:
df.shape

(899164, 27)

In [49]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [50]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [51]:
df.shape

(696350, 27)

In [52]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [53]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
samples_bad_lol['MIS_Status'].value_counts()

MIS_Status
CHGOFF    140687
P I F       4406
Name: count, dtype: int64

In [54]:
status_counts = df.loc[samples_bad_lol.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

print(status_counts)

MIS_Status
CHGOFF    145262
Name: count, dtype: int64


In [55]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)
samples_bad['MIS_Status'].value_counts()

MIS_Status
CHGOFF    145148
Name: count, dtype: int64

In [56]:
samples_bad['MIS_Status'].isna().sum()

0

In [57]:
status_counts = df.loc[samples_bad.index, 'MIS_Status'].value_counts()#.isna().sum() == 0

status_counts

MIS_Status
CHGOFF    145148
Name: count, dtype: int64

In [58]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [59]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [60]:
df.dropna(subset=['BankState', 'Bank', 'State'], inplace=True)

In [61]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [62]:
df['FranchiseCode'] = df['FranchiseCode'].apply(lambda x: 1 if x >= 1 else 0)

In [63]:
df['FranchiseCode'].value_counts()

FranchiseCode
1    493367
0    181033
Name: count, dtype: int64

In [64]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [65]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [66]:
df.shape

(674400, 13)

In [67]:
df.head()

,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv,SBA_Appv,Approve
0,44-45,84,4,2.0,0,0,1,0,N,Y,60000,48000,True
1,72,60,2,2.0,0,0,1,0,N,Y,40000,32000,True
2,62,180,7,1.0,0,0,1,0,N,N,287000,215250,True
5,31-33,120,19,1.0,0,0,1,0,N,N,517000,387750,True
7,81,84,1,2.0,0,0,1,0,N,Y,45000,36000,True


In [68]:
df_to_csv(df, folder_path, 'SBA_Cleaned_maybe.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_maybe.csv


In [69]:
y = df['Approve']
X = df.drop('Approve', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [70]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [71]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

In [72]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [73]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")


In [74]:
param_grid = {
    'model__n_estimators': [70],
    'model__max_depth': [25],
    'model__min_samples_split': [3],
    'model__min_samples_leaf': [80],
    'model__max_features': [None, 'sqrt', 'log2'],
    'model__max_samples': [0.7]
}

In [75]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'GrAppv', 'SBA_Appv'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         Index(['NAICS', 'RevLineCr', 'LowDoc'], dtype='object'))])),
                                       ('model',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=42))]),
             param_grid={'model__max_depth': [25],
                         'model__max_features': [None, 'sqrt', 'log2'],
                         'model__max_samples': [0.7],
                         'model__min_samples_leaf': [80],
                         'model__min_samples_split': [3],
                         'model__n_estimators': [70]},
             scoring='f1_macro')

In [76]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

Meilleurs hyperparamètres trouvés :
{'model__max_depth': 25, 'model__max_features': None, 'model__max_samples': 0.7, 'model__min_samples_leaf': 80, 'model__min_samples_split': 3, 'model__n_estimators': 70}
Meilleure précision trouvée : 0.9058280358051691


In [77]:
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test)

In [78]:
f1_macro = f1_score(y_test, predictions, average='macro')

print("F1-score macro sur l'ensemble de test :", f1_macro)

F1-score macro sur l'ensemble de test : 0.9065352017150367


In [79]:
maybe_leaking_feature = ['RetainedJob', 'CreateJob']
maybe_leaking_fea = ['SBA_Appv', 'GrAppv']
df = df.drop(maybe_leaking_feature, axis=1)

In [80]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[56, 81, 48-49, 44-45, 72]"
1,Term,int64,0,0.0,377,"[84, 120, 108]"
2,NoEmp,int64,0,0.0,499,"[2, 4, 1]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,FranchiseCode,int64,0,0.0,2,"[1, 0]"
5,UrbanRural,int64,0,0.0,3,"[1, 2]"
6,RevLineCr,object,0,0.0,2,"[Y, N]"
7,LowDoc,object,0,0.0,2,[N]
8,GrAppv,int64,0,0.0,16868,"[1000000, 195000, 44000, 205000, 100000]"
9,SBA_Appv,int64,0,0.0,28200,"[37500, 337500, 72250, 9000, 12500]"


In [81]:
y = df['Approve']
X = df.drop('Approve', axis=1)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [83]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [84]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [85]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=42))
])

set_config(transform_output="pandas")

In [86]:
param_grid = {
    'model__n_estimators': [70],
    'model__max_depth': [25],
    'model__min_samples_split': [3],
    'model__min_samples_leaf': [80],
    'model__max_features': [None, 'sqrt', 'log2'],
    'model__max_samples': [0.7]
}

In [87]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('nums',
                                                                         StandardScaler(),
                                                                         Index(['Term', 'NoEmp', 'NewExist', 'FranchiseCode', 'UrbanRural', 'GrAppv',
       'SBA_Appv'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         Index(['NAICS', 'RevLineCr', 'LowDoc'], dtype='object'))])),
                                       ('model',
                                        RandomForestClassifier(n_jobs=-1,
                                                               random_state=42))]),
             param_grid={'model__max_depth': [25],
                         'model__max_features': [None, 'sqrt', 'log2'],
                         'model__max_samples': [0.7],
                         'model__min_samples_leaf': [80],
                         'model__min_samples_split': [3],
                         'model__n_estimators': [70]},
             scoring='f1_macro')

In [88]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

print("Meilleure précision trouvée :", grid_search.best_score_)

Meilleurs hyperparamètres trouvés :
{'model__max_depth': 25, 'model__max_features': None, 'model__max_samples': 0.7, 'model__min_samples_leaf': 80, 'model__min_samples_split': 3, 'model__n_estimators': 70}
Meilleure précision trouvée : 0.9056260242709107


In [89]:
best_model = grid_search.best_estimator_

predictions = best_model.predict(X_test)

In [90]:
f1_macro = f1_score(y_test, predictions, average='macro')

print("F1-score macro sur l'ensemble de test :", f1_macro)

F1-score macro sur l'ensemble de test : 0.9054926582537084
